In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam

In [8]:
# model parameters
reuse = False
block = MEGNet_block
data = '../Data/{}_data_ACSF_expand.pickle'
batch_size = 32
dim = 128
epochs = 20
clip = 2
layer1 = 2
layer2 = 2
factor = 2
lr = 1e-4

#### Train on each type from start

In [3]:
head = MEGNet_block

In [6]:
lr_type =5e-5
epochs_type = 25

In [5]:
for i in range(8):
    # load type data
    data_type = '../Data/{}_data_ACSF_expand_type_'+str(i)+'.pickle'
    train_dl,val_dl = get_data(data_type,batch_size)
    
    # init model
    model = GNN_edgeUpdate(reuse,block,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type)
    
    print('\nTraining type: {}'.format(i))
    model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True)

    save_results(train_loss_perType,val_loss_perType,reuse,block,\
                 head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')

    save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')


Training type: 0
epoch:0, train_loss: +1.045, val_loss: +0.319, 
train_vector: +1.05|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.32|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.589, val_loss: +0.739, 
train_vector: +0.59|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.74|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00



KeyboardInterrupt: 

In [ ]:
    model = GNN_edgeUpdate(reuse,block,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type)
    
    print('\nTraining type: {}'.format(i))
    model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True)



Training type: 0
epoch:0, train_loss: +1.114, val_loss: +0.828, 
train_vector: +1.11|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.83|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.500, val_loss: +0.160, 
train_vector: +0.50|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.16|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: +0.321, val_loss: +0.162, 
train_vector: +0.32|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.16|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: +0.230, val_loss: +0.028, 
train_vector: +0.23|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.03|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: +0.142, val_loss: +0.135, 
train_vector: +0.14|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.13|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:5, train_loss: +0.070, val_loss: -0.087, 
train_vector: +0.07|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.

#### two stage bigger head

In [3]:
lr_type =1e-4/2
epochs_type = 10
file_name = "False_<class 'functions_refactor.schnet_block'>_<class 'functions_refactor.cat3Head'>__data_ACSF_expand.pickle_32_128_2_2_2_2_20_base.tar"

In [4]:
for i in range(8):
    # load type data
    data_type = '../Data/{}_data_ACSF_expand_type_'+str(i)+'.pickle'
    train_dl,val_dl = get_data(data_type,batch_size)
    
    # load base model
    model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    checkpoint = torch.load('../Model/'+file_name)
    model.load_state_dict(checkpoint['model_state_dict'])

    set_requires_grad(model, False)
    model.head = cat3Head_type(dim,8,data_dict[data]['edge_in4']).to('cuda:0')
    
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type) 
    # train head
    print('\nTraining type: {}'.format(i))
    print('\nTraining head')
    model,train_loss_perType,val_loss_perType = train(opt,model,5,train_dl,val_dl,paras,clip,True)
    
    # train all
    set_requires_grad(model, True)
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type)    
    print('\nTraining all')
    model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True,train_loss_perType,val_loss_perType)
    
    save_results(train_loss_perType,val_loss_perType,reuse,block,\
                 cat3Head_type,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_2stage')

    save_model(model,opt,reuse,block,cat3Head_type,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_2stage')


Training type: 0

Training head
epoch:0, train_loss: +0.589, val_loss: +0.061, 
train_vector: +0.59|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.06|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.008, val_loss: +0.214, 
train_vector: +0.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.21|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -0.005, val_loss: +0.116, 
train_vector: -0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.12|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: -0.008, val_loss: +0.121, 
train_vector: -0.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.12|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: -0.010, val_loss: +0.239, 
train_vector: -0.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.24|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

Training completed in 85.70982599258423s

Training all
epoch:5, train_loss: +0.050, val_loss


Training type: 3

Training head
epoch:0, train_loss: -1.242, val_loss: -1.765, 
train_vector: +0.00|+0.00|+0.00|-1.24|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.76|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: -1.703, val_loss: -1.774, 
train_vector: +0.00|+0.00|+0.00|-1.70|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.77|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -1.733, val_loss: -1.693, 
train_vector: +0.00|+0.00|+0.00|-1.73|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.69|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: -1.748, val_loss: -1.865, 
train_vector: +0.00|+0.00|+0.00|-1.75|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.86|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: -1.766, val_loss: -1.651, 
train_vector: +0.00|+0.00|+0.00|-1.77|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.65|+0.00|+0.00|+0.00|+0.00

Training completed in 87.28697752952576s

Training all
epoch:5, train_loss: -1.664, val_loss


Training type: 6

Training head
epoch:0, train_loss: -0.193, val_loss: -0.417, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.19|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.42|+0.00

epoch:1, train_loss: -0.423, val_loss: -0.460, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.42|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.46|+0.00

epoch:2, train_loss: -0.449, val_loss: -0.413, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.45|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.41|+0.00

epoch:3, train_loss: -0.474, val_loss: -0.513, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.47|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.51|+0.00

epoch:4, train_loss: -0.493, val_loss: -0.516, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.49|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.52|+0.00

Training completed in 84.60186243057251s

Training all
epoch:5, train_loss: -0.620, val_loss

### make submissions

In [14]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

From start

In [3]:
head = cat3Head_type

In [4]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'_start')

2Stage

In [3]:
head = cat3Head_type

In [4]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'_2stage')